In [1]:
import pandas as pd 
import numpy as np 
import neo4j

#import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
##from pandas_profiling import ProfileReport


In [2]:
def usuarios_nodo(tx, id, password):
    tx.run("CREATE (:User {id: $id, p: $password})", id=id, password = password)
def peliculas_nodo(tx, nombre, id):
    tx.run("MERGE (m:Movie {nombre: $nombre,id: $id})", nombre=nombre,id= id)
    
def relacion_generos(tx, movie_id, nombre):
    tx.run("MERGE (m:Movie {nombre: $nombre,id: $id})", nombre=nombre,id= id)





In [3]:
data=pd.read_csv('data.tsv',sep='\t')

In [4]:

data.head()
data_1=data.dropna()
data_1.replace("\\N", value = np.nan, inplace = True)
data_1["genres"]=data_1["genres"].replace(np.nan, value = "666")
data_1.infer_objects()

data_1["titleType"]=data_1["titleType"].str.lower()
data_1["primaryTitle"]=data_1["primaryTitle"].str.lower()
data_1["originalTitle"]=data_1["originalTitle"].str.lower()
data_1["genres"]=data_1["genres"].str.lower()
data_1=data_1[data_1["titleType"]=="movie"]


data_1['isAdult']=pd.to_numeric(data_1['isAdult'])
data_1["startYear"]=pd.to_numeric(data_1["startYear"])
data_1["runtimeMinutes"]=pd.to_numeric(data_1["runtimeMinutes"])
data_1["endYear"]=pd.to_numeric(data_1["endYear"])

a=data_1["genres"].unique()
unique_genres = set()

# Iterar sobre cada cadena de géneros en el arreglo
for genres in a:
    # Convertir la cadena en una lista de géneros
    if type(genres) == str:
        genre_list = genres.split(',')
        # Agregar cada género único al conjunto
        unique_genres.update(genre_list)

# Convertir el conjunto de vuelta en una lista
unique_genres_list = list(unique_genres)
print(unique_genres_list)

data_1["tiempo_rank"]=pd.qcut(data_1["runtimeMinutes"], 10,labels = False)
data_1["genres"]=data_1["genres"].astype(str)
data_1['genres'] = [x.strip('()').split(',') for x in data_1['genres']]
data_1.head()


['thriller', 'comedy', 'adventure', 'family', 'animation', 'biography', '666', 'short', 'romance', 'action', 'drama', 'horror', 'sport', 'mystery', 'adult', 'war', 'news', 'history', 'fantasy', 'musical', 'game-show', 'film-noir', 'talk-show', 'music', 'sci-fi', 'documentary', 'western', 'crime', 'reality-tv']


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tiempo_rank
8,tt0000009,movie,miss jerry,miss jerry,0,1894.0,NaN,45.0,[romance],0.0
144,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897.0,NaN,100.0,"[documentary, news, sport]",7.0
498,tt0000502,movie,bohemios,bohemios,0,1905.0,NaN,100.0,[666],7.0
570,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906.0,NaN,70.0,"[action, adventure, biography]",2.0
587,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907.0,NaN,90.0,[drama],5.0


In [5]:
data.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
links=pd.read_csv('links.csv')

In [7]:
peliculas=data_1["tconst"].unique()


In [9]:
links.columns
a=len(links["movieId"].unique())

In [10]:
import random
movies=random.choices(peliculas, k=a)
data_movies = data_1[data_1["tconst"].isin(movies)]

In [11]:
links["imdb"]=movies
class_map=dict(zip(links.movieId,links.imdb))

In [12]:
ratings=pd.read_csv('ratings.csv')
ratings["imdb"]=ratings["movieId"].map(class_map)


In [13]:
seleccionadas=random.choices(movies, k=1000)
usuarios = ratings["userId"].to_list()

usuarios_aleatorios = random.choices(usuarios, k=200)
peliculas_aleatorias = random.choices(seleccionadas, k=200)

# Crear el dataframe con las columnas "usuario" y "pelicula"
visto = pd.DataFrame({'usuario_id': usuarios_aleatorios, 'pelicula_id': peliculas_aleatorias})


In [32]:
from neo4j import GraphDatabase


url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = "ZsZO3ya5YB8EcmVwmxYEqn6Amj_Z7R4t_6VDU5oMIeI"

driver = GraphDatabase.driver(url, auth=(username, password))
#with driver.session() as session:
#    session.run("MATCH (n) DETACH DELETE n")
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")
    

La conexión con Aura Neo4j se ha establecido correctamente. Hay 1163 nodos en la base de datos.


In [15]:
data_movies = data_1[data_1["tconst"].isin(seleccionadas)]
usuarios = visto["usuario_id"].unique()
contraseña=random.choices(range(1000,9999), k= len(usuarios))

In [16]:
demograficos = pd.DataFrame({'usuario_id': usuarios, 'password': contraseña})

In [17]:
resultado = pd.merge(visto, demograficos, on='usuario_id')

In [18]:
resultado

,usuario_id,pelicula_id,password
0,473,tt3391072,5078
1,473,tt0035071,5078
2,592,tt0354053,9487
3,592,tt0057685,9487
4,122,tt2609372,9663
...,...,...,...
195,68,tt17061866,3810
196,246,tt23741924,1869
197,347,tt4550970,4895
198,96,tt4247244,9513


In [19]:
for i in unique_genres_list: 
     with driver.session() as session:
        session.execute_write(
            lambda tx: tx.run("MERGE (m:Genre {nombre: $nombre})", nombre=i)
        )
        


In [36]:
for i in unique_genres_list: 
    print(i)

thriller
comedy
adventure
family
animation
biography
666
short
romance
action
drama
horror
sport
mystery
adult
war
news
history
fantasy
musical
game-show
film-noir
talk-show
music
sci-fi
documentary
western
crime
reality-tv


In [20]:
for index, row in data_movies.iterrows():
    movie_id = row["tconst"]
    title = row["originalTitle"]
    genres = row["genres"] 
    with driver.session() as session:
        session.write_transaction(peliculas_nodo,nombre=title, id = movie_id)

In [21]:
ratings

,userId,movieId,rating,timestamp,imdb
0,1,1,4.0,964982703,tt4460252
1,1,3,4.0,964981247,tt0098992
2,1,6,4.0,964982224,tt12193098
3,1,47,5.0,964983815,tt5884910
4,1,50,5.0,964982931,tt8448696
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,tt21986414
100832,610,168248,5.0,1493850091,tt12698702
100833,610,168250,5.0,1494273047,tt8417690
100834,610,168252,5.0,1493846352,tt2136889


In [22]:
for index, row in resultado.iterrows():
    u_id = row["usuario_id"]
    password = row["password"]
    with driver.session() as session:
        session.write_transaction(usuarios_nodo,id = u_id,password = password)
       
    

In [23]:
resultado

,usuario_id,pelicula_id,password
0,473,tt3391072,5078
1,473,tt0035071,5078
2,592,tt0354053,9487
3,592,tt0057685,9487
4,122,tt2609372,9663
...,...,...,...
195,68,tt17061866,3810
196,246,tt23741924,1869
197,347,tt4550970,4895
198,96,tt4247244,9513


In [27]:
def relacion_generos(tx, movie_id, genre):
    tx.run("MATCH (g:Genre {nombre: $nombre}) "
            "MATCH (m:Movie {id: $id}) "
            "CREATE (m)-[:IN_GENRE]->(g)"
            , nombre=genre, id=movie_id)
    
for index, row in data_movies.iterrows():
    movie_id = row["tconst"]
    title = row["originalTitle"]
    genres = row["genres"]
    for genre in genres:
        with driver.session() as session:
            session.write_transaction(relacion_generos,genre=genre,movie_id= movie_id)
            

In [33]:
def relacion_vistas(tx, movie_id, u_id):
    tx.run("MATCH(u:User {id: $u_id})"
                              "MATCH (m:Movie {id: $id}) "
                              "CREATE (u)-[:IN_VISTAS]->(m)"
            , u_id=u_id, id=movie_id)
for index, row in resultado.iterrows():
    user_id = row["usuario_id"]
    movie_id = row['pelicula_id']      
    with driver.session() as session:    
        session.write_transaction(relacion_vistas,u_id=user_id, movie_id = movie_id)
            
            